<a href="https://colab.research.google.com/github/jasminjahanpuspo/CNN_Architectures_Template/blob/main/K_fold_EnsembleNet(3_rich_model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing import image_dataset_from_directory




In [2]:
import numpy as np
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.preprocessing import image_dataset_from_directory
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [3]:
import numpy as np
import cv2
import os
import pandas as pd

import glob as gb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D ,LeakyReLU
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Average

In [4]:
# Set your parameters
BATCH_SIZE = 32
IMG_SIZE = (224, 224)  # Adjust this according to your model requirements
N_FOLDS = 5

In [5]:
## Define the directories for training, testing, and validation
train_directory = '/content/drive/MyDrive/MIAS/train'
test_directory = '/content/drive/MyDrive/MIAS/test'
valid_directory = '/content/drive/MyDrive/MIAS/val'

train_dataset = image_dataset_from_directory(
    train_directory,
    shuffle=True,
    labels='inferred',
    batch_size=32,
    image_size=(224, 224),
    color_mode='rgb',
    seed=42
)

test_dataset = image_dataset_from_directory(
    test_directory,
    shuffle=False,
    labels='inferred',
    batch_size=32,
    image_size=(224, 224),
    color_mode='rgb',
    seed=42
)

val_dataset = image_dataset_from_directory(
    valid_directory,
    shuffle=True,
    labels='inferred',
    batch_size=32,
    image_size=(224, 224),
    color_mode='rgb',
    seed=42
)


Found 3294 files belonging to 2 classes.
Found 573 files belonging to 2 classes.
Found 573 files belonging to 2 classes.


In [6]:
valid_dataset = image_dataset_from_directory(
    valid_directory,
    shuffle=True,
    labels='inferred',
    batch_size=32,
    image_size=(224, 224),
    color_mode='rgb',
    seed=42
)

Found 573 files belonging to 2 classes.


In [7]:
# Load pre-trained models
model_1 = load_model('/content/drive/MyDrive/MIAS/BC_Code/inceptiov3/inceptionv3-27-0.9721.hdf5')
model_2 = load_model('/content/drive/MyDrive/MIAS/BC_Code/efficientnetb0/efficientnetbo-08-0.9948.hdf5')
model_3 = load_model('/content/drive/MyDrive/MIAS/BC_Code/oldefficientnetb3/efficientnetb3-14-0.9965.hdf5')


##**Meta Learner Logistic Regression**

In [ ]:

# Prepare data (flatten and extract from the dataset for stacking)
X_train = np.concatenate([x for x, _ in train_dataset], axis=0)
y_train = np.concatenate([y for _, y in train_dataset], axis=0)

X_test = np.concatenate([x for x, _ in test_dataset], axis=0)
y_test = np.concatenate([y for _, y in test_dataset], axis=0)


In [ ]:
# Get predictions from base models
def get_model_predictions(model, X):
    return model.predict(X)

In [ ]:
preds_model_1_train = get_model_predictions(model_1, X_train)
preds_model_2_train = get_model_predictions(model_2, X_train)
#preds_model_3_train = get_model_predictions(model_3, X_train)


103/103 [==============================] - 71s 661ms/step


In [ ]:
# For test data
preds_model_1_test = get_model_predictions(model_1, X_test)
preds_model_2_test = get_model_predictions(model_2, X_test)
#preds_model_3_test = get_model_predictions(model_3, X_test)


18/18 [==============================] - 12s 664ms/step


In [ ]:
# Concatenate predictions as features for meta-learner
train_meta_features = np.concatenate(
    [preds_model_1_train, preds_model_2_train ], axis=1
)


In [ ]:
test_meta_features = np.concatenate(
    [preds_model_1_test, preds_model_2_test], axis=1
)


In [ ]:
# Train a simple meta-learner (Logistic Regression or any other ML model)
meta_learner = LogisticRegression()
meta_learner.fit(train_meta_features, y_train)


LogisticRegression()

In [ ]:
# Predict on test data
meta_predictions = meta_learner.predict(test_meta_features)


In [ ]:
# Evaluate the stacking model
test_accuracy = accuracy_score(y_test, meta_predictions)
print(f"Stacking Test Accuracy: {test_accuracy:.4f}")

Stacking Test Accuracy: 0.0140


##**Meta Learner XGBoost**


In [ ]:

pip install xgboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 5.3 MB/s eta 0:00:00


In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
# Create and train the XGBoost meta-learner
meta_learner_xgb = xgb.XGBClassifier(
    n_estimators=200,        # Number of trees
    learning_rate=0.0001,      # Learning rate for boosting
    max_depth=5,             # Maximum depth of the trees
    random_state=42,         # Reproducibility
    use_label_encoder=False, # XGBoost option to avoid label encoder warning
    eval_metric='logloss'    # XGBoost default metric
)

# Fit the meta-learner on the base model predictions
meta_learner_xgb.fit(train_meta_features, y_train)

# Predict on the test set using the XGBoost meta-learner
y_pred_xgb = meta_learner_xgb.predict(test_meta_features)

# Evaluate the XGBoost meta-learner
stacking_test_accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f"Stacking Test Accuracy with XGBoost: {stacking_test_accuracy_xgb:.4f}")


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [18:36:04] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Stacking Test Accuracy with XGBoost: 0.3822


##**Blending**


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score

# Function to get predictions from a model
def get_predictions(model, data):
    predictions = model.predict(data)
    return np.argmax(predictions, axis=1)

# Assuming you have loaded your models: model_1, model_2, model_3

# Generate predictions from each model
predictions_model_1 = get_predictions(model_1, val_dataset)
predictions_model_2 = get_predictions(model_2, val_dataset)

# Assign weights to the models
weights = np.array([0.5, 0.3, 0.2])  # Adjust weights as needed

# Combine predictions
combined_predictions = (
    predictions_model_1 * weights[0] +
    predictions_model_2 * weights[1]
)

# Get final predictions by rounding
final_predictions = np.round(combined_predictions).astype(int)

# Get true labels from the validation dataset
true_labels = []
for _, labels in val_dataset:
    true_labels.extend(labels.numpy())  # Collect all true labels into a list

# Convert true_labels to a numpy array for accuracy calculation
true_labels = np.array(true_labels)

# Calculate accuracy
accuracy = accuracy_score(true_labels, final_predictions)
print(f'Blending Test Accuracy: {accuracy:.4f}')


18/18 [==============================] - 14s 644ms/step
Blending Test Accuracy: 0.5689


##**Voting**

In [ ]:
# Function to get predictions from a model
def get_predictions(model, data):
    predictions = model.predict(data)
    return np.argmax(predictions, axis=1)

# Generate predictions from each model
predictions_model_1 = get_predictions(model_1, val_dataset)
predictions_model_2 = get_predictions(model_2, val_dataset)

# Stack predictions into a single array
all_predictions = np.stack([predictions_model_1, predictions_model_2], axis=0)

# Perform majority voting
final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=all_predictions)

# Get true labels from the validation dataset
true_labels = []
for _, labels in val_dataset:
    true_labels.extend(labels.numpy())  # Collect all true labels into a list

# Convert true_labels to a numpy array for accuracy calculation
true_labels = np.array(true_labels)

# Calculate accuracy
accuracy = accuracy_score(true_labels, final_predictions)
print(f'Voting Test Accuracy: {accuracy:.4f}')

18/18 [==============================] - 13s 666ms/step
Voting Test Accuracy: 0.5672


##**K-fold Voting**

In [ ]:
# Function to get predictions from a model
def get_predictions(model, data):
    predictions = model.predict(data)
    return np.argmax(predictions, axis=1)

# Prepare for K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Store predictions and true labels
all_predictions = []
true_labels = []

# K-Fold Cross-Validation
for train_index, val_index in kf.split(np.arange(len(train_dataset))):
    # Create train and validation datasets based on K-Fold indices
    train_fold = train_dataset.skip(val_index[0]).take(len(train_index))
    val_fold = train_dataset.skip(train_index[0]).take(len(val_index))

    # Get predictions from each model
    predictions_model_1 = get_predictions(model_1, val_fold)
    predictions_model_2 = get_predictions(model_2, val_fold)

    # Stack predictions into a single array
    fold_predictions = np.stack([predictions_model_1, predictions_model_2], axis=0)

    # Perform majority voting
    final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=fold_predictions)

    # Store the predictions and true labels for evaluation
    all_predictions.append(final_predictions)

    # Get true labels from the validation dataset
    for _, labels in val_fold:
        true_labels.extend(labels.numpy())

# Convert true_labels to a numpy array for accuracy calculation
true_labels = np.array(true_labels)

# Calculate accuracy
final_predictions = np.concatenate(all_predictions)
accuracy = accuracy_score(true_labels, final_predictions)
print(f'Voting Test Accuracy: {accuracy:.4f}')

20/20 [==============================] - 14s 662ms/step
Voting Test Accuracy: 0.5167


##**k-fold cross validation**

In [ ]:
# Set your parameters
BATCH_SIZE = 32
IMG_SIZE = (224, 224)  # Adjust this according to your model requirements
N_FOLDS = 5

In [ ]:
# Function to get predictions from a model
def get_predictions(model, data):
    predictions = model.predict(data)
    return np.argmax(predictions, axis=1)

# Prepare for K-Fold Cross-Validation
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

# Store accuracy for each fold
accuracies = []

# K-Fold Cross-Validation
for fold, (train_index, val_index) in enumerate(kf.split(np.arange(len(train_dataset)))):
    print(f'Training on fold {fold + 1}...')

    # Create train and validation datasets based on K-Fold indices
    train_fold = train_dataset.skip(val_index[0]).take(len(train_index))
    val_fold = train_dataset.skip(train_index[0]).take(len(val_index))

    # Get true labels for validation data
    y_val = []
    for _, labels in val_fold:
        y_val.extend(labels.numpy())

    y_val = np.array(y_val)

    # Train model on the training fold (you can train one model or multiple here)
    # Example with model_1:
    history = model_1.fit(
        train_fold,
        epochs=10,  # Adjust epochs as necessary
        validation_data=val_fold
    )

    # Get predictions from the model for the validation fold
    predictions = get_predictions(model_1, val_fold)

    # Calculate accuracy for this fold
    accuracy = accuracy_score(y_val, predictions)
    accuracies.append(accuracy)
    print(f'Fold {fold + 1} Accuracy: {accuracy:.4f}')

# Calculate mean accuracy across all folds
mean_accuracy = np.mean(accuracies)
print(f'Mean K-Fold Accuracy: {mean_accuracy:.4f}')

Training on fold 1...
Epoch 1/10
82/82 [==============================] - 28s 298ms/step - loss: 0.2748 - accuracy: 0.9505 - val_loss: 0.1381 - val_accuracy: 0.9568
Epoch 2/10
82/82 [==============================] - 23s 280ms/step - loss: 0.1733 - accuracy: 0.9447 - val_loss: 0.0171 - val_accuracy: 0.9970
Epoch 3/10
82/82 [==============================] - 24s 284ms/step - loss: 0.0510 - accuracy: 0.9825 - val_loss: 0.0208 - val_accuracy: 0.9926
Epoch 4/10
82/82 [==============================] - 24s 281ms/step - loss: 0.0403 - accuracy: 0.9832 - val_loss: 0.0290 - val_accuracy: 0.9851
Epoch 5/10
82/82 [==============================] - 23s 277ms/step - loss: 0.0338 - accuracy: 0.9874 - val_loss: 0.0163 - val_accuracy: 0.9940
Epoch 6/10
82/82 [==============================] - 24s 288ms/step - loss: 0.0210 - accuracy: 0.9935 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 7/10
82/82 [==============================] - 24s 285ms/step - loss: 0.0212 - accuracy: 0.9912 - val_loss: 0.0021 

In [ ]:
# Calculate test accuracy
# Get true labels for test data
y_test = []
for _, labels in test_dataset:
    y_test.extend(labels.numpy())
y_test = np.array(y_test)

# Get predictions from the model for the test dataset
test_predictions = get_predictions(model_1, test_dataset)

# Calculate test accuracy
test_accuracy = accuracy_score(y_test, test_predictions)
print(f'Test Accuracy: {test_accuracy:.4f}')

18/18 [==============================] - 4s 210ms/step
Test Accuracy: 0.9651


##**K-fold Average Ensemble**

In [8]:
# Function to get predictions from a model
def get_predictions(model, data):
    predictions = model.predict(data)
    return np.argmax(predictions, axis=1)

# Function to get average predictions from multiple models
def get_ensemble_predictions(models, data):
    # Get predictions from each model
    predictions = [model.predict(data) for model in models]

    # Average the predictions
    avg_predictions = np.mean(predictions, axis=0)

    # Convert averaged predictions to class labels
    return np.argmax(avg_predictions, axis=1)


In [9]:
# Assuming train_dataset and val_dataset are preprocessed datasets
# Replace with your own data pipeline
N_FOLDS = 5  # Define number of folds

# Prepare for K-Fold Cross-Validation
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

# Store accuracy for each fold for the ensemble
ensemble_accuracies = []


In [ ]:
# K-Fold Cross-Validation
for fold, (train_index, val_index) in enumerate(kf.split(np.arange(len(train_dataset)))):
    print(f'Training on fold {fold + 1}...')

    # Create train and validation datasets based on K-Fold indices
    train_fold = train_dataset.skip(val_index[0]).take(len(train_index))
    val_fold = train_dataset.skip(train_index[0]).take(len(val_index))

    # Get true labels for validation data
    y_val = []
    for _, labels in val_fold:
        y_val.extend(labels.numpy())

    y_val = np.array(y_val)

    # Train all models on the training fold
    history_model_1 = model_1.fit(
        train_fold,
        epochs=5,  # Adjust epochs as necessary
        validation_data=val_fold,
        verbose=2
    )

    history_model_2 = model_2.fit(
        train_fold,
        epochs=5,
        validation_data=val_fold,
        verbose=2
    )

    history_model_3 = model_3.fit(
        train_fold,
        epochs=5,
        validation_data=val_fold,
        verbose=2
    )

    # Get ensemble predictions for the validation fold
    models = [model_1, model_2, model_3]
    ensemble_predictions = get_ensemble_predictions(models, val_fold)

    # Calculate accuracy for the ensemble model
    ensemble_accuracy = accuracy_score(y_val, ensemble_predictions)
    ensemble_accuracies.append(ensemble_accuracy)

    print(f'Fold {fold + 1} Ensemble Accuracy: {ensemble_accuracy:.4f}')

# Calculate mean accuracy across all folds for the ensemble
mean_ensemble_accuracy = np.mean(ensemble_accuracies)
print(f'Mean K-Fold Ensemble Accuracy: {mean_ensemble_accuracy:.4f}')

Training on fold 1...
21/21 [==============================] - 17s 687ms/step
Fold 1 Ensemble Accuracy: 0.4762
Training on fold 2...


In [10]:
# K-Fold Cross-Validation
for fold, (train_index, val_index) in enumerate(kf.split(np.arange(len(train_dataset)))):
    print(f'Training on fold {fold + 1}...')

    # Create train and validation datasets based on K-Fold indices
    train_fold = train_dataset.skip(val_index[0]).take(len(train_index))
    val_fold = train_dataset.skip(train_index[0]).take(len(val_index))

    # Get true labels for validation data
    y_val = []
    for _, labels in val_fold:
        y_val.extend(labels.numpy())

    y_val = np.array(y_val)

    # Train all models on the training fold
    history_model_1 = model_1.fit(
        train_fold,
        epochs=5,  # Adjust epochs as necessary
        validation_data=val_fold,
        verbose=2
    )

    history_model_2 = model_2.fit(
        train_fold,
        epochs=5,
        validation_data=val_fold,
        verbose=2
    )

    history_model_3 = model_3.fit(
        train_fold,
        epochs=5,
        validation_data=val_fold,
        verbose=2
    )

    # Get ensemble predictions for the validation fold
    models = [model_1, model_2, model_3]
    ensemble_predictions = get_ensemble_predictions(models, val_fold)

    # Calculate accuracy for the ensemble model
    ensemble_accuracy = accuracy_score(y_val, ensemble_predictions)
    ensemble_accuracies.append(ensemble_accuracy)

    print(f'Fold {fold + 1} Ensemble Accuracy: {ensemble_accuracy:.4f}')

# Calculate mean accuracy across all folds for the ensemble
mean_ensemble_accuracy = np.mean(ensemble_accuracies)
print(f'Mean K-Fold Ensemble Accuracy: {mean_ensemble_accuracy:.4f}')

Training on fold 1...
Epoch 1/5
82/82 - 37s - loss: 0.3060 - accuracy: 0.9062 - val_loss: 0.0826 - val_accuracy: 0.9673 - 37s/epoch - 451ms/step
Epoch 2/5
82/82 - 22s - loss: 0.0891 - accuracy: 0.9630 - val_loss: 0.0192 - val_accuracy: 0.9926 - 22s/epoch - 269ms/step
Epoch 3/5
82/82 - 22s - loss: 0.0581 - accuracy: 0.9779 - val_loss: 0.0106 - val_accuracy: 0.9985 - 22s/epoch - 272ms/step
Epoch 4/5
82/82 - 22s - loss: 0.0250 - accuracy: 0.9909 - val_loss: 0.0027 - val_accuracy: 1.0000 - 22s/epoch - 266ms/step
Epoch 5/5
82/82 - 22s - loss: 0.0304 - accuracy: 0.9920 - val_loss: 0.0192 - val_accuracy: 0.9926 - 22s/epoch - 270ms/step
Epoch 1/5
82/82 - 98s - loss: 0.9545 - accuracy: 0.6890 - val_loss: 0.5480 - val_accuracy: 0.6548 - 98s/epoch - 1s/step
Epoch 2/5
82/82 - 83s - loss: 0.4453 - accuracy: 0.8445 - val_loss: 0.0995 - val_accuracy: 0.9613 - 83s/epoch - 1s/step
Epoch 3/5
82/82 - 82s - loss: 0.0797 - accuracy: 0.9775 - val_loss: 0.3644 - val_accuracy: 0.9018 - 82s/epoch - 1s/step
Epo

In [11]:
# Evaluate on the test dataset
models = [model_1, model_2, model_3]
test_predictions = get_ensemble_predictions(models, test_dataset)

# Get true labels for the test dataset
y_test = []
for _, labels in test_dataset:
    y_test.extend(labels.numpy())

y_test = np.array(y_test)

# Calculate accuracy for the test dataset
test_accuracy = accuracy_score(y_test, test_predictions)
print(f'Test Accuracy: {test_accuracy:.4f}')

18/18 [==============================] - 12s 669ms/step
Test Accuracy: 0.9895


In [12]:
# Get true labels for the validation or test set
y_true = []
for _, labels in val_fold:  # Change val_fold to test_dataset for test data
    y_true.extend(labels.numpy())

y_true = np.array(y_true)  # Convert to numpy array for easier processing


In [13]:
def get_ensemble_predictions(models, dataset):
    # Create an empty list to store predictions from all models
    all_predictions = []

    for model in models:
        model_predictions = []
        for images, _ in dataset:  # Loop through dataset without using labels
            predictions = model.predict(images)
            model_predictions.append(predictions)
        all_predictions.append(np.concatenate(model_predictions))

    # Average the predictions from all models (for classification)
    averaged_predictions = np.mean(np.array(all_predictions), axis=0)

    # Convert averaged predictions into class labels (assuming softmax probabilities)
    predicted_labels = np.argmax(averaged_predictions, axis=1)

    return predicted_labels


In [14]:
y_pred = get_ensemble_predictions(models, val_fold)  # or test_dataset for test set


1/1 [==============================] - 1s 585ms/step


In [15]:
from sklearn.metrics import confusion_matrix, classification_report

# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Detailed classification report (Precision, Recall, F1-score)
report = classification_report(y_true, y_pred)
print('Classification Report:')
print(report)


Confusion Matrix:
[[114 224]
 [ 90 212]]
Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.34      0.42       338
           1       0.49      0.70      0.57       302

    accuracy                           0.51       640
   macro avg       0.52      0.52      0.50       640
weighted avg       0.52      0.51      0.49       640



In [22]:
# Initialize counters for correct and incorrect predictions
correct_predictions = 0
incorrect_predictions = 0

# Loop through the validation data to count correct and incorrect predictions
for i, (images, true_label) in enumerate(val_fold):  # or test_dataset
    # Get the true label as an index
    true_label = np.argmax(true_label.numpy())  # Assuming one-hot encoded
    predicted_label = y_pred[i]  # Get predicted label

    # Check if the prediction is correct
    if true_label == predicted_label:
        correct_predictions += 1
    else:
        incorrect_predictions += 1

# Print the counts
print(f'Correct Predictions: {correct_predictions}')
print(f'Incorrect Predictions: {incorrect_predictions}')

# Optionally, print the total predictions and accuracy
total_predictions = correct_predictions + incorrect_predictions
accuracy = correct_predictions / total_predictions * 100 if total_predictions > 0 else 0
print(f'Total Predictions: {total_predictions}')
print(f'Accuracy: {accuracy:.2f}%')


Correct Predictions: 7
Incorrect Predictions: 13
Total Predictions: 20
Accuracy: 35.00%


##**Average Ensemble**

In [ ]:
base_learning_rate = 0.001 # prefferable lr is 0.0001 or 0.001
IMG_SHAPE = IMG_SIZE +(3,)

In [ ]:
models = [model_1, model_2]
model_input = Input(shape=(224, 224, 3))
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble')

In [ ]:
## customize optimizer as Nadam or Adam
ensemble_model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=base_learning_rate),
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                           metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Adjust hyperparameters such as the number of epochs and add early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
#plot bar chart for demonstrating data size
Dataset = []
for folder in os.listdir(train_directory):
    files = gb.glob(pathname=str(train_directory + "/" + folder +"/*.*"))
    Dataset.append(len(files))

In [ ]:
# Calculate total number of samples in the dataset
total = np.sum(Dataset)

# Initialize an empty dictionary to hold the weights
class_weight = {}

# Dynamically calculate the weight for each class
for i in range(len(Dataset)):
    class_weight[i] = (total / (len(Dataset) * Dataset[i]))

# Print the computed weights for each class
for class_id, weight in class_weight.items():
    print(f'Weight for class {class_id}: {weight:.2f}')

# Output the class weights dictionary
print(class_weight)

In [ ]:
## change hyperparameter such as epoches ## EnsembleNet3: Resnet, efficientnet, densenet
history = ensemble_model.fit(train_dataset , verbose=2 , epochs=13 , class_weight=class_weight ,
                               validation_data=valid_dataset , use_multiprocessing= True,callbacks=[early_stopping])

In [ ]:
ensemble_model.evaluate(test_dataset , verbose = 1)